In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sys

In [2]:
def proxy_finder_validate(item, candidates, df1, df2, predictors, orthogonal_vars):

    # validate proxies and st item
    assert item in df1.columns, f'AssertionError: item {item} not in df1.columns'

    assert predictors, f'AssertionError: missing predictors. If you would prefer to not specify predictors, do not pass in a variable.'

    for c in predictors:
        assert c in df1.columns, f'AssertionError: predictor {c} not in df1.columns'
        assert c in df2.columns, f'AssertionError: predictor {c} not in df2.columns' # we need same variable in second dataset
        assert c in df1.select_dtypes(include=['number']).columns, f'predictor {c} is not a numeric column in df1'
        assert c in df2.select_dtypes(include=['number']).columns, f'predictor {c} is not a numeric column in df2'

    for c in candidates:
        assert c in df2.columns, f'AssertionError: candidate {c} not in df2.columns'

    if (orthogonal_vars != None):
        for c in orthogonal_vars:
            assert c in df2.columns, f'AssertionError: orthogonal variable {c} not in df2.columns'


In [3]:
# return a new df that is a copy of df, with: rescale all columns to be
#  between 0 and 1, inclusive. Drop any non-numeric columns. Drop any
# rows that are missing at least one predictor.
def data_rescale(df, predictors):
    df = df.copy() # preserve immutability

    # Select only the numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns

    # drop any rows that are missing at least one predictor
    df = df.dropna(subset=predictors)
    # print('the dataframe we\'re rescaling is size: ') # debug
    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the data and transform it
    scaled_values = scaler.fit_transform(df[numeric_cols])

    # Create a new DataFrame with the scaled values, maintaining the original column names
    scaled_df = pd.DataFrame(scaled_values, columns=numeric_cols, index=df.index)

    return scaled_df

In [17]:
# following class implemented with assistance of Claude AI and reviewed for accuracy
# class MissingDataDensity:
#   def __init__(self, n_components, input_dim):
#     #n_components is how many distributions (typically 5) are used in the GMM
#     self.n_components = n_components
#     self.input_dim = input_dim

#     # mixing weights, means, and standard devs for the GMM model
#     self.pi = nn.Parameter(torch.ones(n_components) / n_components)
#     self.mu = nn.Parameter(torch.randn(n_components, input_dim))
#     self.log_sigma = nn.Parameter(torch.zeros(n_components, input_dim))

#   def get_conditional_params(self, x, missing_matrix):
#     # x is a tensor with shape (batch_size, input_dim)
#     # missing_matrix is 1 where values are missing, 0 otherwise
#     batch_size = x.shape[0]
#     pi = self.pi.expand(batch_size, -1)
#     mu = self.mu.expand(batch_size, -1, -1)
#     sigma = self.log_sigma.exp().expand(batch_size, -1, -1)

#     # Update GMM parameters based on observed values
#     for i in range(batch_size):
#         # ~ is bitwise NOT, inverts the masking array
#         observed = ~missing_matrix[i]
#         if observed.any():
#             # Condition Gaussian parameters on observed values
#             diff = x[i, observed] - mu[i, :, observed]
#             likelihood = torch.exp(-0.5 * (diff**2 / sigma[i, :, observed]).sum(dim=1))
#             pi[i] *= likelihood
#             pi[i] /= pi[i].sum()

#     return pi, mu, sigma

# this function will calculate the ReLU activation (an expected value) over the GMM distribution
# def generalized_relu(w, b, pi, mu, sigma):
#     # w: weight vector
#     # b: bias
#     # pi: GMM mixture coefficients
#     # mu: GMM means
#     # sigma: GMM standard devs

#     # Compute parameters of resulting 1D Gaussian after linear transformation
#     wmu = (w.unsqueeze(1) * mu).sum(dim=2) + b
#     wsigma = torch.sqrt((w.unsqueeze(1)**2 * sigma**2).sum(dim=2))

#     # Implementation of NR function from equation (2) in the paper
#     def nr_function(z):
#         sqrt2 = math.sqrt(2)
#         return (1/sqrt2/math.sqrt(math.pi) * torch.exp(-z**2/2) +
#                 z/2 * (1 + torch.erf(z/sqrt2)))

#     # Compute expected ReLU
#     z = wmu / (wsigma + 1e-8)
#     result = wsigma * nr_function(z)
#     return (pi * result).sum(dim=1)

# class GeneralizedLinear(nn.Module):
#     def __init__(self, in_features, out_features):
#         super().__init__()
#         self.linear = nn.Linear(in_features, out_features)

#     def forward(self, x, missing_mask, density):
#         # For complete data points, use standard linear layer
#         complete_mask = ~missing_mask.any(dim=1)
#         output = torch.zeros(x.shape[0], self.linear.out_features, device=x.device)

#         if complete_mask.any():
#             output[complete_mask] = self.linear(x[complete_mask])

#         # For incomplete data points, compute expected activation
#         if (~complete_mask).any():
#             pi, mu, sigma = density.get_conditional_params(x[~complete_mask],
#                                                          missing_mask[~complete_mask])
#             for j in range(self.linear.out_features):
#                 output[~complete_mask, j] = generalized_relu(
#                     self.linear.weight[j],
#                     self.linear.bias[j],
#                     pi, mu, sigma
#                 )

#         return output

# class MissingDataNN(nn.Module):
#     def __init__(self, input_dim, hidden_dims, output_dim, n_components=5):
#         super().__init__()
#         self.density = MissingDataDensity(n_components, input_dim)

#         # First layer handles missing data
#         self.first_layer = GeneralizedLinear(input_dim, hidden_dims[0])

#         # Regular layers for the rest of the network
#         layers = []
#         for i in range(len(hidden_dims)-1):
#             layers.extend([
#                 nn.Linear(hidden_dims[i], hidden_dims[i+1]),
#                 nn.ReLU()
#             ])
#         layers.append(nn.Linear(hidden_dims[-1], output_dim))

#         self.layers = nn.Sequential(*layers)

#     def forward(self, x, missing_mask):
#         # First layer handles missing data
#         x = self.first_layer(x, missing_mask, self.density)
#         x = F.relu(x)

#         # Rest of the network processes normally
#         return self.layers(x)

In [21]:
# get predictions from the neural network. Takes in
def get_predictions(df_train, df_test, predictors, target, epochs=100, learning_rate=0.001, l2_lambda=0.001):

    # split data for training and testing.
    training_features, validation_features, training_target, validation_target = train_test_split(df_train[predictors].to_numpy(), df_train[target].to_numpy(), test_size=0.2, random_state=42)

    training_features = torch.FloatTensor(training_features)
    training_target = torch.FloatTensor(training_target)
    validation_features = torch.FloatTensor(validation_features)
    validation_target = torch.FloatTensor(validation_target)

    model = nn.Sequential(
        nn.Linear(len(predictors), 64),
        nn.ReLU(),
        nn.Dropout(p=0.5),  
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(32, 1)
    )

    # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    # MSE loss
    loss_func = nn.MSELoss()

    # train the model
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        prediction = model(training_features)
        loss = loss_func(prediction, training_target.unsqueeze(1))

        # Backward pass
        loss.backward()
        optimizer.step()

    # get predictions
    model.eval()
    test_data = torch.FloatTensor(df_test[predictors].to_numpy())

    with torch.no_grad():
        predictions = model(test_data)
        predictions = predictions.numpy().flatten()

        val_predictions = model(validation_features)
        val_predictions = val_predictions.numpy().flatten()


    # exit if correlation between predictions and item is bad
    mse = mean_squared_error(val_predictions, validation_target)
    print(f"Debug statement: Neural Net test MSE = {mse}") ####DEBUG
    if (mse > 0.03):
        print('Input Error: Predictors cannot predict {target} in df1', file=sys.stderr)
        print('Aborting program')
        sys.exit(-1)

    return predictions

In [11]:
 # orthogonalization method
# all data is preprocessed and df test has been appended target preds
def orthogonalize(candidates, df_test, orthogonal_vars):
        orth_scores = {}
        for c in candidates:
            candset = df_test[[c, 'predicted_target']].copy().dropna() # assumes candidate has mostly non-NaN entries
            candcol = candset[c]

            X = sm.add_constant(candcol)
            temp_orth_scores = []
            for orth_var in orthogonal_vars:
                orthset = df_test[[orth_var]].copy().dropna()
                common_indices = candset.index.intersection(orthset.index)
                if common_indices.empty:
                    continue
                orth_col = orthset.loc[common_indices, orth_var]
                if np.var(orth_col) == 0:
                    print("ortho:", orth_var, "candidate", c)
                    continue # zero variance leads to divide by zero error
                candcol_common = candset.loc[common_indices, c]

                X_common = sm.add_constant(candcol_common)
                model = sm.OLS(orth_col, X_common).fit()
                temp_orth_scores.append(model.rsquared)

            if temp_orth_scores:
                orth_scores[c] = sum(temp_orth_scores) / len(temp_orth_scores)
            else:
                orth_scores[c] = 0
        return orth_scores

In [12]:
def proxy_finder(df_train, df_test, target, predictors, num_proxies=1, orth_weight=0.65, candidates=None, orthogonal_vars=None):
    if candidates is None:
        candidates = list(df_test.select_dtypes(include='number').columns) #only numerical data (don't encode categories, make user do that)


    proxy_finder_validate(target, candidates, df_train, df_test, predictors, orthogonal_vars)

    #print(f"Predictors: {predictors}") #DEBUGDEBUGDEBUG------------------------------------------------------------
    #print(f"Candidates: {candidates}")

    # Predict status threat scores in df_test
    df_train = data_rescale(df_train, predictors)
    df_test = data_rescale(df_test, predictors)
    df_train = df_train.dropna(subset=target)

    # Check for NaN entries in the specified columns DEBUG
    #for index, row in df_train.iterrows():
     #   if row[target].isnull().any():
      #      print(f"Entry is NaN in row {index}")

   # print(df_train.head) ## debug
  #  print(df_test.head)
    predicted_scores = get_predictions(df_train, df_test, predictors, target)

    df_test['predicted_target'] = predicted_scores
    #print(f"Predicted scores: {predicted_scores[:10]}")  #DEBUG DEBUG------------------------------------------------------------

    results = {}

    for c in candidates:
        candset = df_test[[c, 'predicted_target']].copy().dropna()
        if candset.empty:
            continue

        pred_scores = candset['predicted_target']
        candcol = candset[c]

        X_pred = sm.add_constant(candcol)
        model_pred = sm.OLS(pred_scores, X_pred).fit()
        results[c] = {
            'R_squared': model_pred.rsquared,
            'p_value': model_pred.pvalues[1],
            'coef': model_pred.params[1]
        }
        #print(f"candidate {c}: Results: {results}")  # Debug statement------------------------------------------------------------

    best_proxies = []

    if orthogonal_vars:
        orth_scores = orthogonalize(candidates, df_test, orthogonal_vars)
        proxy_scores = {}
        for c in candidates:
            try:
                proxy_scores[c] = (c, (1 - orth_weight) * results[c]['R_squared'] - orth_weight * orth_scores[c])
            except KeyError as e:
                continue

        sorted_results = sorted(proxy_scores.values(), key=lambda x: x[1], reverse=True)

        for i in range(min(num_proxies, len(sorted_results))):
            proxy, score = sorted_results[i]
            best_proxies.append(proxy)
            print(f"Proxy {i+1} for {target}: {proxy} with score: {score}")
    else:
        sorted_results = sorted(results.items(), key=lambda x: (-x[1]['R_squared'], x[1]['p_value']))

        for i in range(min(num_proxies, len(sorted_results))):
            proxy, metrics = sorted_results[i]
            best_proxies.append(proxy)
            print(f"Proxy {i+1} for {target}: {proxy} with R_squared: {metrics['R_squared']} and p_value: {metrics['p_value']}")

    return best_proxies

In [13]:
import warnings
warnings.filterwarnings("ignore", message="All-NaN slice encountered")
warnings.filterwarnings("ignore", category=FutureWarning, message="Series.__getitem__ treating keys as positions is deprecated")


# Suppress numpy invalid operation warnings
np.seterr(invalid='ignore')

datafile_train =  "/content/temp_yougov.dta"
datafile_test =  "/content/temp_yougov.dta"
df_train = pd.read_stata(datafile_train)
df_test = pd.read_stata(datafile_test)



In [45]:


target = 'christian_nationalism'  # The target variable in the training set
predictors = [ # predictors in both training and test set
                   'presvote20post',
                   'housevote20post',
                   'senvote20post',
                   'pff_jb',
                   'pff_dt',
                   'pid7',
                   'election_fairnness',
                   'educ',
                   'white',
                   'hispanic',
                   'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new'
                   ]

orthogonal_vars = [
                   'presvote20post',
                   'housevote20post',
                   'senvote20post',
                   'pff_jb',
                   'pff_dt',
                   'pid7',
                   'election_fairnness',
                   'educ',
                   'hispanic',
                   'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new']




best_proxies = proxy_finder(df_train, df_test, target, predictors, orth_weight=0.60, orthogonal_vars=orthogonal_vars, num_proxies=5)
#print(best_proxies)
### orth weight 0.9 --> version, how many interviews, etx
### 0.85 same thing
### 0.8
### 0.75 bad
### 0.7 bad

Debug statement: Neural Net test MSE = 0.017662407927413856
Proxy 1 for christian_nationalism: christian_nationalism with score: 0.10373024819112694
Proxy 2 for christian_nationalism: auth_grid_1 with score: 0.06742502824884508
Proxy 3 for christian_nationalism: auth_grid_3 with score: 0.06482144417052497
Proxy 4 for christian_nationalism: ideo7 with score: 0.06094886255412213
Proxy 5 for christian_nationalism: immigrant_citizenship with score: 0.05407189056011796
